# Two by Two Matrix Jacobians

This notebook emphasizes the multiple views of Jacobians with  examples of 2x2 matrix functions.

In particular we will see the
* Symbolic "vec" format producing 4x4 matrices (generally n² by n² or mn by mn)
* Numerical formats
* The important Linear Transformation view
* Kronecker notation
* An example using ForwardDiff automatic differentiation

We also emphasize that  matrix factorizations are also matrix functions, just as much as the square and the cube.

In [1]:
using Symbolics, LinearAlgebra, ForwardDiff

## Symbolic Matrices

In [2]:
@variables p,q,r,s,θ
X = [p r;q s]

2×2 Matrix{Num}:
 p  r
 q  s

### vec
The `vec` command in Julia and in standard mathematics flattens a matrix column by column.

In [3]:
vec(X)

4-element Vector{Num}:
 p
 q
 r
 s

## The matrix square function $f(X) = X^2$:

In [4]:
X^2

2×2 Matrix{Num}:
 p^2 + q*r  p*r + r*s
 p*q + q*s  q*r + s^2

In [5]:
vec(X^2)

4-element Vector{Num}:
 p^2 + q*r
 p*q + q*s
 p*r + r*s
 q*r + s^2

### Symbolic Jacobian

The Jacobian of the (flattened) matrix function $X^2$ symbolically:

In [6]:
jac(Y,X) =  Symbolics.jacobian(vec(Y),vec(X))

J = jac(X^2, X)

4×4 Matrix{Num}:
 2p      r      q   0
  q  p + s      0   q
  r      0  p + s   r
  0      r      q  2s

In [7]:
### Numerical Jacobian

In [8]:
M = [1 2;3 4]
E = [.0003 .0003;.0002 .0001]
substitute(J,Dict(p=>1,q=>3,r=>2,s=>4))

4×4 Matrix{Int64}:
 2  2  3  0
 3  5  0  3
 2  0  5  2
 0  2  3  8

In [9]:
substitute(J,Dict(p=>1,q=>3,r=>2,s=>4)) * vec(E)

4-element Vector{Float64}:
 0.0019
 0.0022
 0.0023
 0.0021

In [10]:
substitute(J,Dict(p=>1,q=>3,r=>2,s=>4)) * vec(E)

4-element Vector{Float64}:
 0.0019
 0.0022
 0.0023
 0.0021

In [11]:
(M+E)^2 - M^2

2×2 Matrix{Float64}:
 0.00190015  0.00230012
 0.00220008  0.00210007

### Linear Transformation Jacobian 
$$
df = f(X+dX) - f(X) = X dX + dX X = f'(X)[dX]
$$
That is, our Jacobian is the linear transformation $f'(X)$ defined by $E \mapsto \boxed{f'(X)[E] = X E + E X}$.

Notice: there is no flattening; this is just matrix to matrix.

In [12]:
linear_transformation(E) = M*E + E*M

linear_transformation(E)

2×2 Matrix{Float64}:
 0.0019  0.0023
 0.0022  0.0021

## Kronecker product ⊗ notation
Notation that kind of lets you think "flattened" or "not flattened" at the same time.

In [13]:
@variables a,b,c,d
display([p r;q s])
display([a c;b d])

2×2 Matrix{Num}:
 p  r
 q  s

2×2 Matrix{Num}:
 a  c
 b  d

Notice all possible products with the first matrix and the second:

In [14]:
kron([a;b],[p q;r s])

4×2 Matrix{Num}:
 a*p  a*q
 a*r  a*s
 b*p  b*q
 b*r  b*s

In [15]:
kron([a c;b d],[p q;r s] )

4×4 Matrix{Num}:
 a*p  a*q  c*p  c*q
 a*r  a*s  c*r  c*s
 b*p  b*q  d*p  d*q
 b*r  b*s  d*r  d*s

In [16]:
@variables e f g h  🍕 👽 🐼 😸

kron([a b c;d e f],[🍕 👽; 🐼 😸])

4×6 Matrix{Num}:
 a*🍕  a*👽  b*🍕  b*👽  c*🍕  c*👽
 a*🐼  a*😸  b*🐼  b*😸  c*🐼  c*😸
 d*🍕  d*👽  e*🍕  e*👽  f*🍕  f*👽
 d*🐼  d*😸  e*🐼  e*😸  f*🐼  f*😸

It turns out that we can express the Jacobian of the vectorized matrix-square function as
$$
I_2 \otimes X + X^T \otimes I_2
$$

In [17]:
I2 = [1 0; 0 1]
display(kron(I2,X) + kron(X',I2))
J

4×4 Matrix{Num}:
 2p      r      q   0
  q  p + s      0   q
  r      0  p + s   r
  0      r      q  2s

4×4 Matrix{Num}:
 2p      r      q   0
  q  p + s      0   q
  r      0  p + s   r
  0      r      q  2s

In [18]:
kron([🍕 👽; 🐼 😸],I2)

4×4 Matrix{Num}:
 🍕   0  👽   0
  0  🍕   0  👽
 🐼   0  😸   0
  0  🐼   0  😸

In [19]:
kron(I2,[🍕 👽; 🐼 😸])

4×4 Matrix{Num}:
 🍕  👽   0   0
 🐼  😸   0   0
  0   0  🍕  👽
  0   0  🐼  😸

In [20]:
kron(I2,X)

4×4 Matrix{Num}:
 p  r  0  0
 q  s  0  0
 0  0  p  r
 0  0  q  s

In [21]:
kron(X',I2)

4×4 Matrix{Num}:
 p  0  q  0
 0  p  0  q
 r  0  s  0
 0  r  0  s

### Key Kronecker identity

$$(A \times B) \,  \mathrm{vec}(C) =  \mathrm{vec}(BCA^T)$$

In [22]:
A = rand(5,7)
B = rand(4,3)
C = rand(3,7)
kron(A,B) * vec(C) ≈ vec(B*C*A')

true

In [23]:
kron( rand(5,5) , rand(5,5) )

25×25 Matrix{Float64}:
 0.344355    0.401632   0.336621     …  0.467098     0.0831696   0.0246478
 0.0545394   0.417074   0.0616338       0.0855235    0.631118    0.308202
 0.0836655   0.0498243  0.167332        0.232192     0.54969     0.207034
 0.143303    0.0619499  0.000187419     0.000260064  0.446035    0.405095
 0.0129838   0.0525151  0.117099        0.162487     0.0269203   0.3691
 0.320094    0.373336   0.312906     …  0.103006     0.0183409   0.00543545
 0.050697    0.387691   0.0572917       0.01886      0.139177    0.067966
 0.0777712   0.0463142  0.155544        0.0512039    0.12122     0.0456559
 0.133207    0.0575855  0.000174215     5.73504e-5   0.0983616   0.0893332
 0.0120691   0.0488153  0.108849        0.0358323    0.00593657  0.0813955
 0.184895    0.215649   0.180743     …  0.415281     0.0739434   0.0219136
 0.029284    0.223941   0.0330932       0.0760362    0.561106    0.274012
 0.0449228   0.0267523  0.0898463       0.206434     0.488711    0.184067
 0.0769441

### Useful Krockecker identities

* $(A\otimes B)^T=A^T\otimes B^T$
* $(A\otimes B)^{-1}=A^{-1}\otimes B^{-1}$
* $\det(A\otimes B)=\det(A)^m\det(B)^n$, $A\in\Re^{n,n}, B\in\Re^{m,m}$ 
* $\mathrm{trace}(A\otimes B)=\mathrm{trace}(A) \, \mathrm{trace}(B)$
* $A\otimes B$ is orthogonal if $A$ and $B$ are orthogonal
* $(A \otimes B)(C \otimes D)=(AC) \otimes (BD)$
* If $Au = \lambda u$, and $Bv=\mu v$, then if $X=vu^T$, then
  $BXA^T =\lambda \mu X$, and also $AX^T B^T =
  \lambda \mu X^T$.  Therefore $A \otimes B$ and $B \otimes A$
  have the same eigenvalues, and transposed eigenvectors.

(See [Wikipedia](https://en.wikipedia.org/wiki/Kronecker_product#Properties) for more properties. )

## The vectorized Jacobian in Kronecker notation


You see $$(I \otimes X + X^T \otimes I)  \operatorname{vec}(dX) = \operatorname{vec}(XdX + dX X) = \operatorname{vec}( d(X^2))  $$

(br)
showing that $\boxed{d(X^2) = (I \otimes X + X^T \otimes I) dX}$.

Sometimes it is nice to think of $I \otimes X + X^T \otimes I$ as a linear operator on *non-vectorized* matrices.   We denote this by square brackets: $(I \otimes X + X^T \otimes I) [E] = XE + EX$, or more generally by $(A \otimes B) [C] = BCA^T$.

## Automatic Differentiation (is not finite differences nor symbolic)
It comes in forward and reverse modes. Let's try forward.

In [24]:
J

4×4 Matrix{Num}:
 2p      r      q   0
  q  p + s      0   q
  r      0  p + s   r
  0      r      q  2s

In [25]:
ForwardDiff.jacobian(X->X^2,M)

4×4 Matrix{Int64}:
 2  2  3  0
 3  5  0  3
 2  0  5  2
 0  2  3  8

In [26]:
substitute(J, Dict(X.=>[1 3;2 4] ))

4×4 Matrix{Int64}:
 2  3  2  0
 2  5  0  2
 3  0  5  3
 0  3  2  8

In [27]:
ForwardDiff.jacobian(X->X^2,X)

4×4 Matrix{Num}:
 2p      r      q   0
  q  p + s      0   q
  r      0  p + s   r
  0      r      q  2s

## The matrix cube function $X \mapsto X^3$

In [28]:
expand.(X^3)

2×2 Matrix{Num}:
                p^3 + 2p*q*r + q*r*s  (p^2)*r + p*r*s + q*(r^2) + r*(s^2)
 (p^2)*q + p*q*s + (q^2)*r + q*(s^2)                 p*q*r + 2q*r*s + s^3

### Symbolic Jacobian
The Jacobian of the (flattened) matrix function $X^3$ symbolically:

In [29]:
expand.(jac(X^3, X))

4×4 Matrix{Num}:
 3(p^2) + 2q*r              2p*r + r*s              2p*q + q*s            q*r
    2p*q + q*s  p^2 + p*s + 2q*r + s^2                     q^2     p*q + 2q*s
    2p*r + r*s                     r^2  p^2 + p*s + 2q*r + s^2     p*r + 2r*s
           q*r              p*r + 2r*s              p*q + 2q*s  2q*r + 3(s^2)

In [30]:
expand.(ForwardDiff.jacobian(X->X^3,X))

4×4 Matrix{Num}:
 3(p^2) + 2q*r              2p*r + r*s              2p*q + q*s            q*r
    2p*q + q*s  p^2 + p*s + 2q*r + s^2                     q^2     p*q + 2q*s
    2p*r + r*s                     r^2  p^2 + p*s + 2q*r + s^2     p*r + 2r*s
           q*r              p*r + 2r*s              p*q + 2q*s  2q*r + 3(s^2)

## Linear transformation Jacobian

By the product rule, $d(X^3) = dX \, X^2 + X \, dX \, X + X^2 \, dX$.

with numerical data:

In [31]:
X³′(X, dX) = dX * X^2 + X * dX * X + X^2 * dX

X³′ (generic function with 1 method)

In [32]:
(E+M)^3 - M^3

2×2 Matrix{Float64}:
 0.0129015  0.0161016
 0.0174014  0.0195013

In [33]:
X³′(M, E)

2×2 Matrix{Float64}:
 0.0129  0.0161
 0.0174  0.0195

check against the symbolic answer:

In [34]:
substitute( Symbolics.jacobian(vec(X^3), vec(X)) , Dict(p=>M[1,1],q=>M[2,1],r=>M[1,2],s=>M[2,2]))

4×4 Matrix{Int64}:
 15  12  18   6
 18  33   9  27
 12   4  33  18
  6  18  27  60

In [35]:
substitute( Symbolics.jacobian(vec(X^3), vec(X)) , Dict(p=>M[1,1],q=>M[2,1],r=>M[1,2],s=>M[2,2])) * vec(E)

4-element Vector{Float64}:
 0.012899999999999998
 0.0174
 0.0161
 0.0195

### The Jacobian in Kronecker Notation

Again using the key Kronecker identity, we find that:
$$
\operatorname{vec}\left( dX \, X^2 + X \, dX \, X + X^2 \, dX \right) =
\left( (X^T)^2 \otimes I + X^T \otimes X + I \otimes X^2 \right) \operatorname{vec}(dX)
$$
giving the Jacobian $\boxed{dX \, X^2 + X \, dX \, X + X^2 \, dX}$, or in the $2 \times 2$ case:

In [36]:
expand.( kron(X'^2,I2) +  + kron(X',X) + kron(I2,X^2)  )

4×4 Matrix{Num}:
 3(p^2) + 2q*r              2p*r + r*s              2p*q + q*s            q*r
    2p*q + q*s  p^2 + p*s + 2q*r + s^2                     q^2     p*q + 2q*s
    2p*r + r*s                     r^2  p^2 + p*s + 2q*r + s^2     p*r + 2r*s
           q*r              p*r + 2r*s              p*q + 2q*s  2q*r + 3(s^2)

# The LU Decomposition

Recall the LU Decomposition $A = LU$ factors a matrix into unit lower-triangular $L$ and upper triangular $U$:

In [37]:
L,U = lu(X);
display(L)
U

2×2 Matrix{Num}:
     1  0
 q / p  1

2×2 Matrix{Num}:
 p               r
 0  s + (-q*r) / p

In [38]:
simplify_fractions.(L*U)

2×2 Matrix{Num}:
 p  r
 q  s

The four entries of X: p,q,r,s are transformed into these four entries in LU:

In [39]:
[L[2,1],U[1,1],U[1,2],U[2,2]]

4-element Vector{Num}:
          q / p
              p
              r
 s + (-q*r) / p

In [40]:
jac([L[2,1],U[1,1],U[1,2],U[2,2]], X)

4×4 Matrix{Num}:
      -(q / (p^2))     1 / p         0  0
                 1         0         0  0
                 0         0         1  0
 -((-q*r) / (p^2))  (-r) / p  (-q) / p  1

Exercise: Relate this to $d(LU) = dL U + L dU$

## Traceless symmetric eigenproblem: an example with two parameters, not four

In [41]:
S = [p s; s -p]

2×2 Matrix{Num}:
 p   s
 s  -p

We know that the eigenvalues add to 0 (from the trace) and the eigenvectors are orthogonal (from being symmetric), so we can represent the eigenvectors and eigenvalues:

In [42]:
Q = [cos(θ/2) -sin(θ/2); sin(θ/2) cos(θ/2)]  # Eigenvector matrix

2×2 Matrix{Num}:
 cos((1//2)*θ)  -sin((1//2)*θ)
 sin((1//2)*θ)   cos((1//2)*θ)

In [43]:
Λ = [r 0;0 -r] # Eigenvalue matrix

2×2 Matrix{Num}:
 r   0
 0  -r

In [44]:
Symbolics.simplify.(Q * Λ * Q')

2×2 Matrix{Num}:
 r*(cos((1//2)*θ)^2) - r*(sin((1//2)*θ)^2)  …                                    r*sin(θ)
                                  r*sin(θ)     -r*(cos((1//2)*θ)^2) + r*(sin((1//2)*θ)^2)

The relationship between θ,r and p,s:

In [45]:
S

2×2 Matrix{Num}:
 p   s
 s  -p

In [46]:
simplify.(Q*Λ*Q')

2×2 Matrix{Num}:
 r*cos(θ)   r*sin(θ)
 r*sin(θ)  -r*cos(θ)

In [47]:
[r*cos(θ) r*sin(θ) ; r*sin(θ) -r*cos(θ)]

2×2 Matrix{Num}:
 r*cos(θ)   r*sin(θ)
 r*sin(θ)  -r*cos(θ)

In [48]:
simplify.(jac( (Q*Λ*Q')[1:2] ,  [r,θ]))

2×2 Matrix{Num}:
 cos(θ)  -r*sin(θ)
 sin(θ)   r*cos(θ)

Interesting mathematical observation: these are the formulas you may remember
from other classes that relate cartesian coordinates to polar coordinates in the plane.

In [49]:
jacobian_det = simplify(det(simplify.(jac( (Q*Λ*Q')[1:2] ,  [r,θ]))))

r*(cos(θ)^2) + r*(sin(θ)^2)

Mathematical aside det J=r , this is the change of variables from x,y to r,θ that you may have seen in 18.02.  This eigenvalue problem is the same as the cartesian coordinates to polar representations of the plane. Often written dx dy = r dr dθ

## The full 2x2 symmetric eigenproblem

In [50]:
@variables λ₁ λ₂

2-element Vector{Num}:
 λ₁
 λ₂

We think of diagonalization:
$$
\left( \begin{array}{cc}
p & s \\ s & r 
\end{array} \right) =
\left( \begin{array}{rr} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) 
\end{array} \right)
\left( \begin{array}{cc}
\lambda_1 & 0 \\ 0  & \lambda_2 
\end{array} \right) 
\left( \begin{array}{rr} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) 
\end{array} \right)^T 
$$
that is, $S = QΛQ^T$, as the function
$$
\lambda_1,\lambda_2,θ  \mapsto  p,r,s
$$


In [51]:
Q = [cos(θ) -sin(θ); sin(θ) cos(θ)]
S = Q*[λ₁ 0;0 λ₂]*Q'
[p s;s r], S
J = jac([S[1,1],S[2,2],S[1,2]] , [λ₁,λ₂,θ])

3×3 Matrix{Num}:
      cos(θ)^2  …                         -2cos(θ)*sin(θ)*λ₁ + 2cos(θ)*sin(θ)*λ₂
      sin(θ)^2                             2cos(θ)*sin(θ)*λ₁ - 2cos(θ)*sin(θ)*λ₂
 cos(θ)*sin(θ)     (cos(θ)^2)*λ₁ - (cos(θ)^2)*λ₂ - (sin(θ)^2)*λ₁ + (sin(θ)^2)*λ₂

The determinant of this transformation simplifies to $\lambda_1 - \lambda_2$
which some people interpret as a kind of repulsion between the two eigenvalues:
that is there is a tendency for the two eigenvalues to not want to be too close
together.  (If both are equal, when n=2, the matrix is $\alpha I$, one condition
takes three parameters down to 1)